# 第四章　暖冷房設備

## 第一節　全般

1．適用範囲～4．記号及び単位は省略

### 5．暖房方式及び冷房方式

省略

### 6．暖房設備の一次エネルギー消費量及び処理負荷と未処理負荷

#### 6.1 処理負荷及び未処理負荷

#### 6.1.1 住戸全体を連続的に暖房する方式

$$
\large
Q_{T,H,A,d,t} = min(Q_{max,H,d,t}, L_{H,A,d,t})
$$

$$
\large
Q_{UT,H,A,d,t} = L_{H,A,d,t} - Q_{T,H,A,d,t}
$$

$Q_{T,H,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備の処理暖房負荷(MJ/h)  
$Q_{UT,H,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷(MJ/h)  
$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の最大暖房出力(MJ/h)  
$L_{H,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房負荷の合計(MJ/h)  

$L_{H,A,d,t}$が0未満の場合は0とする。 

$$
\large
L_{H,A,d,t} = \sum_{i=1～12}L_{H,d,t,i} 
$$

#### 6.1.2 居室のみを暖房する方式

$$
\large
Q_{T,H,MR,d,t} = Q_{T,H,d,t,i}|_{i=1}
$$

$$
\large
Q_{UT,H,MR,d,t} = Q_{UT,H,d,t,i}|_{i=1}
$$

$Q_{T,H,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備の処理暖房負荷(MJ/h)  
$Q_{UT,H,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備の未処理暖房負荷(MJ/h)  
$Q_{T,H,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画𝑖に設置された暖房設備機器等の処理暖房負荷(MJ/h)  
$Q_{UT,H,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画𝑖に設置された暖房設備機器等の未処理暖房負荷(MJ/h)  

$$
\large
Q_{T,H,OR,d,t} = \sum_{i=2～5} Q_{T,H,d,t,i} 
$$

$$
\large
Q_{UT,H,OR,d,t} = \sum_{i=2～5} Q_{UT,H,d,t,i} 
$$

$Q_{T,H,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備の処理暖房負荷(MJ/h)  
$Q_{UT,H,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備の未処理暖房負荷(MJ/h)  

$$
\large
Q_{T,H,d,t,i} = min(Q_{max,H,d,t,i}, L_{H,d,t,i})
$$

$$
\large
Q_{UT,H,d,t,i} = L_{H,d,t,i} - Q_{T,H,d,t,i}
$$

$Q_{max,H,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器等の最大暖房出力(MJ/h)  
$L_{H,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの暖房負荷(MJ/h)  

$L_{H,d,t,i}$が0未満の場合は0とする。 

#### 6.2 暖房設備のエネルギー消費量

$$
\large
E_{E,H,d,t} = E_{E,H,hs,d,t} + E_{E,H,aux,ass,d,t}
$$

$$
\large
E_{G,H,d,t} = E_{G,H,hs,d,t}
$$

$$
\large
E_{K,H,d,t} = E_{K,H,hs,d,t}
$$

$$
\large
E_{M,H,d,t} = E_{M,H,hs,d,t}
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の消費電力量(kWh/h)  
$E_{G,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備のガス消費量(MJ/h)  
$E_{K,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の灯油消費量(MJ/h)  
$E_{M,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等の消費電力量(kWh/h)  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等のガス消費量(MJ/h)  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等の灯油消費量(MJ/h)  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,aux,ass,d,t}$：日付$d$の時刻$t$における1時間当たりの空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分(kWh/h)  

日付$d$の時刻$t$における1 時間当たりの空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分$E_{E,H,aux,ass,d,t}$は、空気集熱式太陽熱利用設備を採用しない場合はゼロとする。

#### 6.3 暖房設備機器のエネルギー消費量 

#### 6.3.1 住戸全体を連続的に暖房する方式

$$
\large
E_{E,H,hs,d,t} = E_{E,H,hs,A,d,t}
$$

$$
\large
E_{G,H,hs,d,t} = E_{G,H,hs,A,d,t}
$$

$$
\large
E_{K,H,hs,d,t} = E_{K,H,hs,A,d,t}
$$

$$
\large
E_{M,H,hs,d,t} = E_{M,H,hs,A,d,t}
$$

$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

#### 6.3.2 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合

$$
\large
E_{E,H,hs,d,t} = E_{E,H,hs,MROR,d,t}
$$

$$
\large
E_{G,H,hs,d,t} = E_{G,H,hs,MROR,d,t}
$$

$$
\large
E_{K,H,hs,d,t} = E_{K,H,hs,MROR,d,t}
$$

$$
\large
E_{M,H,hs,d,t} = E_{M,H,hs,MROR,d,t}
$$

$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備の消費電力量(kWh/h)  
$E_{G,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備のガス消費量(MJ/h)  
$E_{K,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備の灯油消費量(MJ/h)  
$E_{M,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備のその他の燃料による一次エネルギー消費量(MJ/h)  

#### 6.3.3 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合 

$$
\large
E_{E,H,hs,d,t} = E_{E,H,hs,MR,d,t} + E_{E,H,hs,OR,d,t}
$$

$$
\large
E_{G,H,hs,d,t} = E_{G,H,hs,MR,d,t} + E_{G,H,hs,OR,d,t}
$$

$$
\large
E_{K,H,hs,d,t} = E_{K,H,hs,MR,d,t} + E_{K,H,hs,OR,d,t}
$$

$$
\large
E_{M,H,hs,d,t} = E_{M,H,hs,MR,d,t} + E_{M,H,hs,OR,d,t}
$$

$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

$$
\large
E_{E,H,hs,MR,d,t} = E_{E,H,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{G,H,hs,MR,d,t} = E_{G,H,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{K,H,hs,MR,d,t} = E_{K,H,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{M,H,hs,MR,d,t} = E_{M,H,hs,d,t,i}|_{i=1}
$$

$E_{E,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

$$
\large
E_{E,H,hs,OR,d,t} = \sum_{i=2～5} E_{E,H,hs,d,t,i} 
$$

$$
\large
E_{G,H,hs,OR,d,t} = \sum_{i=2～5} E_{G,H,hs,d,t,i} 
$$

$$
\large
E_{K,H,hs,OR,d,t} = \sum_{i=2～5} E_{K,H,hs,d,t,i} 
$$

$$
\large
E_{M,H,hs,OR,d,t} = \sum_{i=2～5} E_{M,H,hs,d,t,i} 
$$

#### 6.4 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値 

#### 6.4.1 住戸全体を連続的に暖房する方式

$$
\large
E_{UT,H,d,t} = Q_{UT,H,A,d,t} \times \alpha_{UT,H,A}
$$

$E_{UT,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値(MJ/h)  
$Q_{UT,H,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷(MJ/h)  
$\alpha_{UT,H,A}$：住戸全体を連続的に暖房する方式における暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数  

<div style="text-align: center;font-weight: bold;">表　未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数</div>

|地域の区分|住戸全体を連続的に暖房する方式|
|-|-|
|1|1.61|
|2|1.46|
|3|1.32|
|4|1.30|
|5|1.20|
|6|1.09|
|7|1.12|
|8|‐|

#### 6.4.2 居室のみを暖房する方式

$$
\large
E_{UT,H,d,t} = Q_{UT,H,MR,d,t} \times \alpha_{UT,H,MR} + Q_{UT,H,OR,d,t} \times \alpha_{UT,H,OR}
$$

$E_{UT,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値(MJ/h)  
$Q_{UT,H,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備の未処理暖房負荷(MJ/h)  
$Q_{UT,H,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備の未処理暖房負荷(MJ/h)  
$\alpha_{UT,H,MR}$：居室のみを暖房する方式における主たる居室に設置された暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数  
$\alpha_{UT,H,OR}$：居室のみを暖房する方式におけるその他の居室に設置された暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数  

<div style="text-align: center;font-weight: bold;">表　未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数</div>

<table>
<thead>
<tr>
<th rowspan="3">地域の区分</th>
<th colspan="4">居室のみを暖房する方式</th>
</tr>
<tr>
<th colspan="2">主たる居室</th>
<th colspan="2">その他の居室</th>
</tr>
<tr>
<th>連続運転</th>
<th>間歇運転</th>
<th>連続運転</th>
<th>間歇運転</th>
</tr>
</thead>
<tbody>
<tr><th>1</th><td>1.59</td><td>1.21</td><td>1.59</td><td>1.22</td></tr>
<tr><th>2</th><td>1.66</td><td>1.22</td><td>1.66</td><td>1.24</td></tr>
<tr><th>3</th><td>1.63</td><td>1.22</td><td>1.63</td><td>1.23</td></tr>
<tr><th>4</th><td>1.60</td><td>1.21</td><td>1.60</td><td>1.23</td></tr>
<tr><th>5</th><td>1.53</td><td>1.05</td><td>1.53</td><td>1.04</td></tr>
<tr><th>6</th><td>1.57</td><td>0.96</td><td>1.57</td><td>1.00</td></tr>
<tr><th>7</th><td>1.63</td><td>1.01</td><td>1.63</td><td>1.34</td></tr>
<tr><th>8</th><td>-</td><td>-</td><td>-</td><td>-</td></tr>
</tbody>
</table>

#### 6.5 コージェネレーション設備又は住棟セントラル暖房設備が賄う温水暖房の熱負荷等

省略

### 7．冷房設備の一次エネルギー消費量及び処理負荷と未処理負荷

#### 7.1 処理負荷及び未処理負荷

#### 7.1.1 住戸全体を連続的に冷房する方式

$$
\large
Q_{T,CS,A,d,t} = min(Q_{max,CS,d,t}, L_{CS,A,d,t})
$$

$$
\large
Q_{UT,CS,A,d,t} = L_{CS,A,d,t} - Q_{T,CS,A,d,t}
$$

$Q_{T,CS,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の処理冷房顕熱負荷(MJ/h)  
$Q_{UT,CS,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の未処理冷房顕熱負荷(MJ/h)  
$Q_{max,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の最大冷房顕熱出力(MJ/h)  
$L_{CS,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷の合計(MJ/h)

$$
\large
Q_{T,CL,A,d,t} = min(Q_{max,CL,d,t}, L_{CL,A,d,t})
$$

$$
\large
Q_{UT,CL,A,d,t} = L_{CL,A,d,t} - Q_{T,CL,A,d,t}
$$

$Q_{T,CL,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の処理冷房潜熱負荷(MJ/h)  
$Q_{UT,CL,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の未処理冷房潜熱負荷(MJ/h)  
$Q_{max,C,d,tL}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の最大冷房潜熱出力(MJ/h)  
$L_{CL,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房潜熱負荷の合計(MJ/h)

$L_{CS,A,d,t}$が0未満の場合は0とする。$L_{CL,A,d,t}$が0未満の場合は0とする。 

$$
\large
L_{CS,A,d,t} = \sum_{i=1～12} L_{CS,d,t,i} 
$$

$$
\large
L_{CL,A,d,t} = \sum_{i=1～12} L_{CL,d,t,i} 
$$

$L_{CS,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房顕熱負荷(MJ/h)  
$L_{CL,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房潜熱負荷(MJ/h)  

#### 7.1.2 居室のみを冷房する方式

$$
\large
Q_{T,CS,MR,d,t} = Q_{T,CS,d,t,i}|_{i=1}
$$

$$
\large
Q_{T,CL,MR,d,t} = Q_{T,CL,d,t,i}|_{i=1}
$$

$$
\large
Q_{UT,CS,MR,d,t} = Q_{UT,CS,d,t,i}|_{i=1}
$$

$$
\large
Q_{UT,CL,MR,d,t} = Q_{UT,CL,d,t,i}|_{i=1}
$$

$Q_{T,CS,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の処理冷房顕熱負荷(MJ/h)  
$Q_{T,CL,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の処理冷房潜熱負荷(MJ/h)  
$Q_{UT,CS,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の未処理冷房顕熱負荷(MJ/h)  
$Q_{UT,CL,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の未処理冷房潜熱負荷(MJ/h)  
$Q_{T,CS,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の処理冷房顕熱負荷(MJ/h)  
$Q_{T,CL,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の処理冷房潜熱負荷(MJ/h)  
$Q_{UT,CS,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の未処理冷房顕熱負荷(MJ/h)  
$Q_{UT,CL,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の未処理冷房潜熱負荷(MJ/h)

$$
\large
Q_{T,CS,OR,d,t} = \sum_{i=2～5} Q_{T,CS,d,t,i} 
$$

$$
\large
Q_{T,CL,OR,d,t} = \sum_{i=2～5} Q_{T,CL,d,t,i} 
$$

$$
\large
Q_{UT,CS,OR,d,t} = \sum_{i=2～5} Q_{UT,CS,d,t,i} 
$$

$$
\large
Q_{UT,CL,OR,d,t} = \sum_{i=2～5} Q_{UT,CL,d,t,i} 
$$

$Q_{T,CS,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の処理冷房顕熱負荷(MJ/h)  
$Q_{T,CL,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の処理冷房潜熱負荷(MJ/h)  
$Q_{UT,CS,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の未処理冷房顕熱負荷(MJ/h)  
$Q_{UT,CL,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の未処理冷房潜熱負荷(MJ/h)  

$$
\large
Q_{T,CS,d,t,i} = min(Q_{max,CS,d,t,i}, L_{CS,d,t,i})
$$

$$
\large
Q_{UT,CS,d,t,i} = L_{CS,d,t,i} - Q_{T,CS,d,t,i}
$$

$Q_{max,CS,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の最大冷房顕熱出力(MJ/h)  
$L_{CS,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房顕熱負荷(MJ/h)  

$$
\large
Q_{T,CL,d,t,i} = min(Q_{max,CL,d,t,i}, L_{CL,d,t,i})
$$

$$
\large
Q_{UT,CL,d,t,i} = L_{CL,d,t,i} - Q_{T,CL,d,t,i}
$$

$Q_{max,CL,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の最大冷房潜熱出力(MJ/h)  
$L_{CL,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房潜熱負荷(MJ/h)  

$L_{CS,d,t,i}$が0未満の場合は0とする。また、$L_{CL,d,t,i}$が0未満の場合は0とする。 

#### 7.2 冷房設備のエネルギー消費量

$$
\large
E_{E,C,d,t} = E_{E,C,hs,d,t}
$$

$$
\large
E_{G,C,d,t} = E_{G,C,hs,d,t}
$$

$$
\large
E_{K,C,d,t} = E_{K,C,hs,d,t}
$$

$$
\large
E_{M,C,d,t} = E_{M,C,hs,d,t}
$$

$E_{E,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の消費電力量(kWh/h)  
$E_{G,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備のガス消費量(MJ/h)  
$E_{K,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の灯油消費量(MJ/h)  
$E_{M,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

#### 7.3 冷房設備機器のエネルギー消費量

#### 7.3.1 住戸全体を連続的に冷房する方式

$$
\large
E_{E,C,hs,d,t} = E_{E,C,hs,A,d,t}
$$

$$
\large
E_{G,C,hs,d,t} = E_{G,C,hs,A,d,t}
$$

$$
\large
E_{K,C,hs,d,t} = E_{K,C,hs,A,d,t}
$$

$$
\large
E_{M,C,hs,d,t} = E_{M,C,hs,A,d,t}
$$

$E_{E,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

#### 7.3.2 居室のみを冷房する方式

$$
\large
E_{E,C,hs,d,t} = E_{E,C,hs,MR,d,t} +  E_{E,C,hs,OR,d,t}
$$

$$
\large
E_{G,C,hs,d,t} = E_{G,C,hs,MR,d,t} +  E_{G,C,hs,OR,d,t}
$$

$$
\large
E_{K,C,hs,d,t} = E_{K,C,hs,MR,d,t} +  E_{K,C,hs,OR,d,t}
$$

$$
\large
E_{M,C,hs,d,t} = E_{M,C,hs,MR,d,t} +  E_{M,C,hs,OR,d,t}
$$

$E_{E,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

$$
\large
E_{E,C,hs,MR,d,t} = E_{E,C,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{G,C,hs,MR,d,t} = E_{G,C,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{K,C,hs,MR,d,t} = E_{K,C,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{M,C,hs,MR,d,t} = E_{M,C,hs,d,t,i}|_{i=1}
$$

$E_{E,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

$$
\large
E_{E,C,hs,OR,d,t} = \sum_{i=2～5} E_{E,C,hs,d,t,i} 
$$

$$
\large
E_{G,C,hs,OR,d,t} = \sum_{i=2～5} E_{G,C,hs,d,t,i} 
$$

$$
\large
E_{K,C,hs,OR,d,t} = \sum_{i=2～5} E_{K,C,hs,d,t,i} 
$$

$$
\large
E_{M,C,hs,OR,d,t} = \sum_{i=2～5} E_{M,C,hs,d,t,i} 
$$

## 付録 A

省略

## 付録 B

省略